<a href="https://colab.research.google.com/github/adalves-ufabc/2021.QS-PLN/blob/main/2021_Q1_PLN_Notebook_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2021.Q1]**
Prof. Alexandre Donizeti Alves

# **Classificação de Texto com spaCy**

Uma tarefa comum em PLN é a **classificação de texto**. Isso é "classificação" no sentido convencional de aprendizado de máquina, porém é aplicada a texto. Os exemplos incluem detecção de *spam*, análise de sentimento etc.

Neste exemplo, veremos como fazer a classificação de texto usando a biblioteca **`spaCy`**. O **`spaCy`** é uma biblioteca de processamento de linguagem natural (para Python) que tem desde funcionalidades "básicas" como tokenização, que consiste em um pré-processamento do texto, até mais complexas que permitem treinar modelos estatísticos para classificação de textos.

Neste exemplo, o classificador detectará mensagens de spam, uma funcionalidade comum na maioria dos clientes de e-mail. Aqui está uma visão geral dos dados que usaremos:

In [1]:
import pandas as pd

# dataset: https://www.kaggle.com/matleonard/nlp-course

# loading the spam data
# ham is the label for non-spam messages
spam = pd.read_csv('/content/spam_kaggle.csv', encoding='latin-1')
spam.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


***Bag of Words***

Os modelos de aprendizado de máquina não aprendem com dados de texto brutos. Em vez disso, você precisa converter o texto em algo numérico.

A representação mais simples é uma variação da codificação *One-Hot*. Você representa cada documento como um vetor de frequências de termos para cada termo no vocabulário. O vocabulário é construído a partir de todos os tokens (termos) no corpus (a coleção de documentos).

Como exemplo, tome as frases "Chá é vida. Chá é amor." e "Chá é saudável, calmante e delicioso." como nosso corpus. O vocabulário então é {"chá", "é", "vida", "amor", "saudável", "calmante", "e", "delicioso"} (ignorando a pontuação).

Para cada documento, conte quantas vezes um termo ocorre e coloque essa contagem no elemento apropriado de um vetor. A primeira frase tem "chá" duas vezes e essa é a primeira posição em nosso vocabulário, então colocamos o número 2 no primeiro elemento do vetor. Nossas frases como vetores se parecem com:

> v1=[2  2  1  1  0  0  0  0]

> v2=[1  1  0  0  1  1  1  1]

Isso é chamado de representação do ***bag of words*** (sacola de palavras). Você pode ver que documentos com termos semelhantes terão vetores semelhantes. Os vocabulários freqüentemente têm dezenas de milhares de termos, então esses vetores podem ser muito grandes.

### **Construindo um modelo *Bag of Words***

Depois de ter seus documentos em uma representação *Bag of Words*, você pode usar esses vetores como entrada para qualquer modelo de aprendizado de máquina. **`spaCy`** lida com a conversão de palavras e construindo um modelo linear simples para você com a classe `TextCategorizer`.

O *`TextCategorizer`* é um pipe spaCy. Pipes são classes para processamento e transformação de tokens. Quando você cria um modelo spaCy com `nlp = spacy.load ('en_core_web_sm')`, há pipes default que realizam marcação de parte da fala (*part of speech tagging*), reconhecimento de entidade e outras transformações. 

>
Quando você executa o texto por meio de um modelo `doc = nlp ("Algum texto aqui")`, a saída dos pipes são anexados aos tokens no objeto `doc`. Os lemas para `token.lemma_` vêm de um desses pipes.

Você pode remover ou adicionar pipes aos modelos. O que faremos aqui é criar um modelo vazio sem pipes (exceto um tokenizador, uma vez que todos os modelos sempre têm um tokenizador). Em seguida, criaremos um pipe `TextCategorizer` e o adicionaremos ao modelo vazio.

In [3]:
import spacy

# create an empty model
nlp = spacy.blank("en")

# create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe("textcat",
                          config={"exclusive_classes": True, "architecture": "bow"})

# the TextCategorizer to the empty model
nlp.add_pipe(textcat)

Como as classes são de *ham* ou *spam*, definimos "exclusive_classes" como `True`. Também configuramos com a arquitetura *Bag of Words*" ("bow"). 

Em seguida, adicionaremos os rótulos ao modelo. Aqui, "*ham*" são para mensagens reais, "*spam*" são mensagens de spam.

In [4]:
# add labels to text classifier
textcat.add_label("ham")
textcat.add_label("spam")

1

### **Treinando um modelo categorizador de texto**

Em seguida, você converterá os rótulos nos dados para o formulário que o `TextCategorizer` requer. Para cada documento, você criará um dicionário de valores booleanos para cada classe.

Por exemplo, se um texto for "ham", precisamos de um dicionário {'ham': True, 'spam': False}. O modelo está procurando por esses rótulos em outro dicionário com a chave '*cats*' (categorias).

In [5]:
train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} 
                for label in spam['label']]

In [6]:
# then we combine the texts and labels into a single list
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

Agora você está pronto para treinar o modelo. Primeiro, crie um otimizador usando `nlp.begin_training()`. **`spaCy`** usa esse otimizador para atualizar o modelo. Em geral, é mais eficiente treinar modelos em pequenos lotes. **`spaCy`** fornece a função de `minibatch` que retorna um gerador de *minibatches* para treinamento. Finalmente, os *minibatches* são divididos em textos e rótulos, e então usados com `nlp.update` para atualizar os parâmetros do modelo.

In [7]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) but we need to
    # send separate lists for texts and labels to update().
    # This is a quick way to split a list of tuples into lists
    texts, labels = zip(*batch)
    nlp.update(texts, labels, sgd=optimizer)

Este é apenas um loop de treinamento (ou época) através dos dados. O modelo normalmente precisará de várias épocas. Use outro loop para mais épocas e, opcionalmente, embaralhe novamente os dados de treinamento no início de cada loop.

In [8]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoca in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 0.4309954632938684}
{'textcat': 0.6465297238214021}
{'textcat': 0.7822909041447552}
{'textcat': 0.8687101021063368}
{'textcat': 0.9246688244620618}
{'textcat': 0.9618610965655705}
{'textcat': 0.9899194243909888}
{'textcat': 1.008498932771929}
{'textcat': 1.0229856369099788}
{'textcat': 1.0330240397877635}


### **Fazendo predições**

Agora que você tem um modelo treinado, pode fazer predições com o método `predict()`. O texto de entrada precisa ser tokenizado com `nlp.tokenizer`. Em seguida, você passa os tokens para o método de previsão que retorna as ponderações (*score*). As ponderações são a probabilidade do texto de entrada pertencer às classes.

In [9]:
texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA" ]
docs = [nlp.tokenizer(text) for text in texts]
    
# use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[9.9994600e-01 5.4017470e-05]
 [1.2052479e-02 9.8794752e-01]]


As ponderações são usadas para prever uma única classe ou rótulo, escolhendo o rótulo com a maior probabilidade. Você obtém o índice de maior probabilidade com `scores.argmax` e, em seguida, usa o índice para obter o rótulo da string com `textcat.labels`.

In [10]:
# from the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['ham', 'spam']


Avaliar o modelo é simples, uma vez que você tem as predições. Para medir a acurácia, calcule quantas predições corretas são feitas em alguns dados de teste, dividido pelo número total de predições.